# Day-6

#RUL Target Computation & Documentatio

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Suppress warnings for clearer output
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 1. Load rolling windows sequence (NumPy array)
sequences = np.load('/content/drive/MyDrive/Infosys_Internship/Data Preparation/rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequences.shape}")

Loaded rolling window sequences with shape: (17631, 30, 66)


In [12]:
# 2. Load sequence metadata (engine_id, cycle)
metadata_df = pd.read_csv('/content/drive/MyDrive/Infosys_Internship/Data Preparation/sequence_metadata.csv')
print(f"Loaded sequence metadata with shape: {metadata_df.shape}")
metadata_df.head()

Loaded sequence metadata with shape: (17631, 2)


,engine_id,cycle
0,1,31.0
1,1,32.0
2,1,33.0
3,1,34.0
4,1,35.0


In [13]:
# 3. Calculate Remaining Useful Life (RUL) per sequence

# Find max cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,192.0
1,2,287.0
2,3,179.0
3,4,189.0
4,5,269.0


In [17]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()

,engine_id,cycle,max_cycle_x,max_cycle_y,max_cycle
0,1,31.0,192.0,192.0,192.0
1,1,32.0,192.0,192.0,192.0
2,1,33.0,192.0,192.0,192.0
3,1,34.0,192.0,192.0,192.0
4,1,35.0,192.0,192.0,192.0


In [18]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [19]:
metadata_df.head()

,engine_id,cycle,max_cycle_x,max_cycle_y,max_cycle,RUL
0,1,31.0,192.0,192.0,192.0,161.0
1,1,32.0,192.0,192.0,192.0,160.0
2,1,33.0,192.0,192.0,192.0,159.0
3,1,34.0,192.0,192.0,192.0,158.0
4,1,35.0,192.0,192.0,192.0,157.0


In [20]:
# Optional: Cap RUL values
rul_cap = 130
metadata_df['RUL'] = metadata_df['RUL'].clip(upper=rul_cap)

# Drop max_cycle column if not needed further
metadata_df.drop(columns=['max_cycle'], inplace=True)

metadata_df.head()

,engine_id,cycle,max_cycle_x,max_cycle_y,RUL
0,1,31.0,192.0,192.0,130.0
1,1,32.0,192.0,192.0,130.0
2,1,33.0,192.0,192.0,130.0
3,1,34.0,192.0,192.0,130.0
4,1,35.0,192.0,192.0,130.0


In [21]:
# 4. Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3]  # first 3 engines
for engine in engines_to_check:
    sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
    print(f"\nEngine {engine} - Cycles and RULs:")
    print(sample.head(10))
    print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle    RUL
0   31.0  130.0
1   32.0  130.0
2   33.0  130.0
3   34.0  130.0
4   35.0  130.0
5   36.0  130.0
6   37.0  130.0
7   38.0  130.0
8   39.0  130.0
9   40.0  130.0
     cycle  RUL
152  183.0  9.0
153  184.0  8.0
154  185.0  7.0
155  186.0  6.0
156  187.0  5.0
157  188.0  4.0
158  189.0  3.0
159  190.0  2.0
160  191.0  1.0
161  192.0  0.0

Engine 2 - Cycles and RULs:
     cycle    RUL
162   31.0  130.0
163   32.0  130.0
164   33.0  130.0
165   34.0  130.0
166   35.0  130.0
167   36.0  130.0
168   37.0  130.0
169   38.0  130.0
170   39.0  130.0
171   40.0  130.0
     cycle  RUL
409  278.0  9.0
410  279.0  8.0
411  280.0  7.0
412  281.0  6.0
413  282.0  5.0
414  283.0  4.0
415  284.0  3.0
416  285.0  2.0
417  286.0  1.0
418  287.0  0.0

Engine 3 - Cycles and RULs:
     cycle    RUL
419   31.0  130.0
420   32.0  130.0
421   33.0  130.0
422   34.0  130.0
423   35.0  130.0
424   36.0  130.0
425   37.0  130.0
426   38.0  130.0
427   39.0  130.0
428   4

In [25]:
# 5. Save updated metadata with RUL target
metadata_df.to_csv('/content/drive/MyDrive/Infosys_Internship/Data Preparation/sequence_metadata_with_RUL.csv', index=False)
print("Saved updated metadata with RUL as '/content/drive/MyDrive/Infosys_Internship/Data Preparation/sequence_metadata_with_RUL.csv'")

Saved updated metadata with RUL as '/content/drive/MyDrive/Infosys_Internship/Data Preparation/sequence_metadata_with_RUL.csv'


## Process Steps:

- Loaded rolling window feature sequences from 'rolling_windows_feature.npy'.
- Loaded sequence metadata from 'sequence_metadata.csv' including engine and cycle identifiers.
- Computed Remaining Useful Life (RUL) for each sequence as the difference between maximum cycle for the engine and the current cycle.
- Applied RUL capping at 130 to stabilize regression targets.
- Saved updated metadata including RUL targets for downstream model training.
## Outputs:

- Rolling window sequences (NumPy array) ready for modeling.
- Updated sequence metadata with assigned RUL targets.
## Next Steps:

- Use these datasets for training and evaluating LSTM/GRU predictive models for RUL.